In [1]:
%%capture
!pip install transformers
import pandas as pd
from transformers import pipeline

# Initialize the fill-mask pipeline
unmasker = pipeline('fill-mask', model='microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')

In [2]:
unmasker = pipeline('fill-mask', 
                    model='emilyalsentzer/Bio_ClinicalBERT',
                    tokenizer='emilyalsentzer/Bio_ClinicalBERT')

In [ ]:
import pandas as pd

# Set the number of rows to read
num_rows = int(pd.read_excel('/content/bert cm - Copy - Copy (2).xlsx').shape[0] * 1.0)

# Read the first 1% of rows
disease_df = pd.read_excel('/content/bert cm - Copy - Copy (2).xlsx', nrows=num_rows)

# Print the first few rows to verify the data was loaded correctly
print(disease_df.head())

In [4]:
disease_df

,ICD10,disease
0,A01.00,"Typhoid fever, unspecified"
1,A01.01,Typhoid meningitis


In [10]:
#obsolete
import psutil
from tqdm import tqdm

# Create a list of all possible disease names
disease_list = disease_df['disease']

# Initialize an empty dictionary to store the filled-in sentences
sentences_dict = {}

# Calculate the total number of iterations needed for the progress bar
total_iterations = len(disease_list) * 20

# Initialize the progress bar
pbar = tqdm(total=total_iterations)

# Generate filled-in sentences for all diseases
for disease in disease_list:
    sentence = f" for the {disease}, the first line of testing and investigations is usually the current procedural terminology CPT code [MASK]"
    filled_sentences = unmasker(sentence, top_k=20)
    if disease not in sentences_dict:
        sentences_dict[disease] = {f"Drug {j+1}": "" for j in range(20)}
    for i in range(len(filled_sentences)):
        if i >= 20:
            break
        if sentences_dict[disease][f"Drug {i+1}"] == "":
            drug_name = filled_sentences[i]['token_str']
            probability = filled_sentences[i]['score']
            sentences_dict[disease][f"Drug {i+1}"] = f"{drug_name} ({probability})"
        # Update the progress bar
        pbar.update(1)
        
        # Check memory usage and print a warning if RAM usage exceeds 90%
        ram_usage = psutil.virtual_memory().percent
        if ram_usage > 90:
            print("Warning: RAM usage exceeds 90%")

# Create a new dataframe from the dictionary of filled-in sentences
result_df = pd.DataFrame.from_dict(sentences_dict, orient='index')

# Add the 'ICD10' column from the input dataframe to the output dataframe
result_df['ICD10'] = disease_df['ICD10']

# Print the resulting dataframe
print(result_df)

# Close the progress bar
pbar.close()

100%|██████████| 40/40 [00:00<00:00, 58.99it/s]

                                            Drug 1                   Drug 2  \
Typhoid fever, unspecified   . (0.817108154296875)  - (0.07572146505117416)   
Typhoid meningitis          . (0.8219215869903564)  - (0.06833580136299133)   

                                                   Drug 3  \
Typhoid fever, unspecified  status (0.022615989670157433)   
Typhoid meningitis              is (0.017024755477905273)   

                                                   Drug 4  \
Typhoid fever, unspecified       ) (0.010262362658977509)   
Typhoid meningitis          status (0.016830608248710632)   

                                               Drug 5  \
Typhoid fever, unspecified  is (0.007600924000144005)   
Typhoid meningitis           ) (0.010791422799229622)   

                                                Drug 6  \
Typhoid fever, unspecified    , (0.006048062350600958)   
Typhoid meningitis          was (0.006265833042562008)   

                                               

In [11]:
result_df

,Drug 1,Drug 2,Drug 3,Drug 4,Drug 5,Drug 6,Drug 7,Drug 8,Drug 9,Drug 10,...,Drug 12,Drug 13,Drug 14,Drug 15,Drug 16,Drug 17,Drug 18,Drug 19,Drug 20,ICD10
"Typhoid fever, unspecified",. (0.817108154296875),- (0.07572146505117416),status (0.022615989670157433),) (0.010262362658977509),is (0.007600924000144005),", (0.006048062350600958)",; (0.0047003235667943954),: (0.004645689390599728),was (0.004098658449947834),in (0.0026213284581899643),...,( (0.0019374970579519868),for (0.0018440253334119916),and (0.0016606455901637673),list (0.0016558831557631493),since (0.0011934149079024792),of (0.0011855706106871367),at (0.0010608222801238298),# (0.0009388089529238641),as (0.0006677780183963478),NaN
Typhoid meningitis,. (0.8219215869903564),- (0.06833580136299133),is (0.017024755477905273),status (0.016830608248710632),) (0.010791422799229622),was (0.006265833042562008),: (0.00446392223238945),; (0.003546688938513398),for (0.0035407966934144497),", (0.003304117126390338)",...,on (0.002315249526873231),of (0.001763576758094132),( (0.0017068925080820918),at (0.0016092057339847088),and (0.0015558769227936864),# (0.001322446158155799),since (0.0012132971314713359),list (0.0008094439399428666),as (0.00078888627467677),NaN
